# BEE 4750 Homework 5: Solid Waste Disposal

**Name**: Jonathan Marcuse

**ID**: jrm564

> **Due Date**
>
> Friday, 11/10/23, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will formulate, solve, and analyze a standard
    generating capacity expansion problem.
-   In Problem 2, you will add a CO<sub>2</sub> constraint to the
    capacity expansion problem and identify changes in the resulting
    solution.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

``` julia
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
```

``` julia
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables
```

In [77]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/Desktop/BEE4750-2/hw05-jrmarcuse`


In [78]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Background

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided in the table below.

|    **Disposal Facility**     | **Capacity** (Mg/d) | **Fixed cost** (\$/d) | **Tipping Fee ** (\$/Mg) | **Recycling Cost** (\$/Mg) |
|:-------------:|:-------------:|:-------------:|:----------:|:--------------:|
|           Landfill           |         200         |         2000          |            50            |                            |
| Materials Recycling Facility |         350         |         1500          |            7             |    40 (per Mg recycled)    |
|   Waste-to-Energy Facility   |         210         |         2500          |            60            |                            |

Transportation costs are \$1.5/Mg-km, and the relative distances between
the cities and facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

|   **Component**   | **% of total mass** | **Combustion ash** (%) | **MRF Recycling rate** (%) |
|:---------------------:|:--------------:|:---------------:|:---------------:|
|    Food Wastes    |         15          |           8            |             0              |
| Paper & Cardboard |         40          |           7            |             55             |
|     Plastics      |          5          |           5            |             15             |
|     Textiles      |          3          |           10           |             10             |
|  Rubber, Leather  |          2          |           15           |             0              |
|       Wood        |          5          |           2            |             30             |
|    Yard Wastes    |         18          |           2            |             40             |
|       Glass       |          4          |          100           |             60             |
|      Ferrous      |          2          |          100           |             75             |
|     Aluminum      |          2          |          100           |             80             |
|    Other Metal    |          1          |          100           |             50             |
|   Miscellaneous   |          3          |           70           |             0              |

The information in the above table will help you determine the overall
recycling and ash fractions. Note that the recycling residuals, which
may be sent to either landfill or the WTE, have different ash content
than the ash content of the original MSW. You will need to determine
these fractions to construct your mass balance constraints.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

## Problems (Total: 40 Points)

### Problem 1 (22 points)

In this problem, you will develop an optimal disposal plan for the two
cities.

In [79]:
#Loading in Data from above
Disposal_Facility = [200 2000 50 0; 350 1500 7 40; 210 2500 60 0];
City_Facility = [5 30 15; 15 25 10; 13 45 20; 0 32 18; 32 0 15; 18 15 0];
Component = [15 8 0; 40 7 55; 5 5 15; 3 10 10; 2 15 0; 5 2 30; 18 2 40; 4 100 60; 2 100 75; 2 100 80; 1 100 50; 3 70 0];

#### Problem 1.1 (3 points)

Based on the information above, calculate the overall recycling and ash
fractions for the waste produced by each city.

In [80]:
#Overall recycling percent will be the weighted average of the recycling rates
recyc_fraction = sum(0.01*Component[:,1].*Component[:,3]);
#Overall ash fraction
    # Total mass component times combustion ash percent plus the total mass component times the percent not recycled time combustion ash percent
ash_fraction = sum(0.01*Component[:,1].*Component[:,2])+sum(0.01*Component[:,1].*(0.01*[100 - i for i in Component[:,3]]).*Component[:,2]);

#### Problem 1.2 (2 points)

What are the decision variables for your optimization problem? Provide
notation and variable meaning.

My decision variables will be a 3x3 matrix of waste transported values connecting all cities to all disposals. Each city connects to 3 disposals and there are 3 cities which is why it will be a matrix W of size 3x3 where in Wij i is the city number and j is the disposal number.

Next the residual waste transported from disposal k to disposal j will be a matrix R of size 3x3 where in Rkj k is the first disposal number and j is the first disposal number. In this matrix there will be a constraint that states that when k=j R=0 as there is no transportation within the same disposal site, and then the transportation from the landfill to MRF and WTE will be 0 as the landfill is an end destination. And finally, the transportation from WTE to MRF will also be zero as you cannot recycle waste that has already been combusted in the MRF

Finally a binary decision variable for if the disposal facility is in use must be implemented to determine if fixed cost must be included. This will be variable Yj for j in 1:3 where j is the disposal number and Y is 1 if any waste is transported into the disposal facility, and 0 if no waste is transported in.

To summarize the decision variables are Wij for i in 1:3 and j in 1:3; Rkj for k in 1:3 and j in 1:3, and Yj for j in 1:3.
i is the city number 1:3 and j is the disposal facility number 1:3 where 1 is LF, 2 is MRF and 3 is WTE.

#### Problem 1.3 (3 points)

Formulate the objective function. Make sure to include any needed
derivations or justifications for your equation(s).

The objective function will be the total transportation, disposal and recycling costs summed up. The transportation cost will be the transportation cost rate (a) multiplied by the distance (l) between locations multiplied by the mass of waste transported (W) or sum(a\*l\*W). This will also apply to the transportation between disposal facilities so we will need a distance matrix for distances between disposal facilities (lr) and then the same equation applies - sum(a\*lr\*R). The Disposal cost will be the fixed cost of operating the disposal (c) facility multiplied by the decision variabel to operate the disposal facility (Y) plus the sum of the tipping fee or variable cost per mass transported (b) multiplied by the waste transported (W) as well as the waste transported between disposal facilities R or to simplify: sum(c\*Y)+sum(b\*W)+sum(b\*W). Finally the recyclinc cost will be the total amount of waste transported into the waste recycling facility (sum(W[:,2])) multiplied by the recycing fraction multiplied by the cost of recycling or to summarize sum(W[:,2])\*recyc_fraction\*recyc_cost.

To put it all together:

Total Cost = sum(a\*l\*W)+sum(a\*l\*R)+sum(c\*Y)+sum(b\*W)+sum(b\*R)+sum(W[:,2])\*recyc_fraction\*recyc_cost

The final simplified objective function given by the model is as follow:

$57.5 W[1,1] + 72.5 W[2,1] + 69.5 W[3,1] + 67.1 W[1,2] + 59.6 W[2,2] + 89.6 W[3,2] + 82.5 W[1,3] + 75 W[2,3] + 90 W[3,3] + 98 R[2,1] + 77 R[3,1] + 55 R[1,2] + 29.5 R[3,2] + 87 R[1,3] + 82.5 R[2,3] + 2000 Y[1] + 1500 Y[2] + 2500 Y[3] + 50 R[1,1] + 7 R[2,2] + 60 R[3,3]$

#### Problem 1.4 (4 points)

Derive all relevant constraints. Make sure to include any needed
justifications or derivations.

The constraint for the amount of ash transported from the waste to energy to landfill will be as follows: the ash fraction multiplied by the sum of all inputs into the WTE disposal facility or - ash_fraction\*(W13+W23+W33+R23)

The constraint for the mass balance of waste at the recycling facility will be the total amount of waste leaving the recycling facility will be equal to one minus the recycling fraction multiplied by the mass of the total inputs to the recycling facility or - R21+R23 = (1-recyc_fraction)\*(W12+W22+W32)

The general mass balance constraints will have the total waste leaving each city must be equal to the total production of solid waste from each city or - sum(W[i,:])=Si for i in 1:3 where i is the city number.

The commitments constraint (Y) will be a binary constraint that states whether or not the disposal facility is being operated. The value will be 0 only if the sum of all waste streams entering the disposal facility is 0, and otherwise it will be 1 or Yi = 0 if sum(W[i,:])=0, else=1 for i in 1:3.

The capacity constraints for the disposal facility will state that the sum of all waste streams into the disposal facility will be less than or equal to that facilities capacity or - sum(W[:,j])\<=Kj for j in 1:3

The residuals that must be zero constraint will take the sum of all the R values that must be zero which is 6 out of the 9 of them and set that equal to zero or - R11+R22+R33+R12+R13+R31=0

Finally the non-negative constraint will state that W\>=0 and R\>=0

#### Problem 1.5 (3 points)

Implement your optimization problem in `JuMP`.

In [88]:
s = [100 90 120]; #vector of waste produced in each city in Mg/day
k = Disposal_Facility[:,1]; #Vector of capacities of each disposal facility
l = City_Facility[1:3,:]; #matrix of transportation distances between cities and disposals
lr = City_Facility[4:6,:]; #matrix of transportation distances between disposal facilities
a = 1.5; #transport cost rate in $/Mg-km
c = Disposal_Facility[:,2]; #fixed cost of disposal operation $/day
b = [50 7 60]; #variable cost of disposal from tipping fee $/Mg
recyc_cost = Disposal_Facility[2,4]; #Cost in $/Mg for recycling solid waste
M = 1000000; #M value for Big-M reformulation for Y constraint
I = 1:3; #city numbered city 1,2&3
J = 1:3; #disposal number where 1 is LF, 2 is MRF & 3 is WTE
K = 1:3; #disposal secondary number for residuals where 1 is LF, 2 is MRF & 3 is WTE

#initiate model
waste_mod = Model(HiGHS.Optimizer)

#initiate decision variables
@variable(waste_mod,W[i in I, j in J]>=0); #waste transported between cities and disposal facilities
@variable(waste_mod,R[k in K, j in J]>=0); #residual waste transported between disposal facilities
@variable(waste_mod,Y[j in J],binary=true); #binary variable for if disposal facility is being operated

#Formulate the objective function to minimize total cost
    #Transportation costs + Disposal costs + Recycling costs
@objective(waste_mod,Min,sum(a*l.*W)+sum(a*lr.*R)+sum(c.*Y)+sum(b.*W)+sum(b.*R)+sum(W[:,2])*0.01*recyc_fraction*recyc_cost); 

#Add all constraints
@constraint(waste_mod, Ash, R[3,1]==0.01*ash_fraction*(sum(W[:,3])+R[2,3])); #ash fraction to landfill from WTE
@constraint(waste_mod, Rcyc, R[2,1]+R[2,3]==(1-0.01*recyc_fraction)*sum(W[:,2])); #waste in and out mass balance at MRF
@constraint(waste_mod, Mass_Bal[i in I], sum(W[i,:])==s[i]); #mass balance of total waste leaving each city equal to waste produced by each city
@constraint(waste_mod,commit[j in J],M*Y[j]>=sum(W[:,j])); #If the variable Y is 0 then the sum of waste streams into the disposal facility must be 0
@constraint(waste_mod,cap[j in J],sum(W[:,j])<=k[j]); #total inputs into disposal facilities must be less than the capacity
@constraint(waste_mod, residuals,R[1,1]+R[2,2]+R[3,3]+R[1,2]+R[1,3]+R[3,2]==0); #All of these residuals must be zero

optimize!(waste_mod)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
9 rows, 13 cols, 31 nonzeros
8 rows, 12 cols, 27 nonzeros

Solving MIP model with:
   8 rows
   12 cols (3 binary, 0 integer, 0 implied int., 9 continuous)
   27 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   0               27870.76095      100.00%        0      0      0         0     0.0s
         0       0         0   0.00%   25548.180305    27870.76095        8.33%        0      0      0         4     0.0s

Solving report
  Status            Optimal
  Primal bound      27870.76095
  Dual bound        27870.76095
  Gap               0%

#### Problem 1.6 (2 points)

Find the optimal solution. Report the optimal objective value.

In [108]:
obj = round.(objective_value(waste_mod);digits=2)
println("The objective function value is \$$obj")

The objective function value is $27870.76


#### Problem 1.7 (5 points)

Draw a diagram showing the flows of waste between the cities and the
facilities. Which facilities (if any) will not be used? Does this
solution make sense?

![Diagram of Waste Flow](IMG-1015.jpg)

In [110]:
display(value.(Y))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:3
And data, a 3-element Vector{Float64}:
  1.0
 -0.0
  1.0

The Materials Recycling Facility will not be used which makes sense as all of the waste can go direct to the landfill or waste to energy without the model incurring additional recycling costs for waste. There should be added constraints to account for the environmental benefits of recycling.

### Problem 2 (18 points)

It is projected that in the near future the state will introduce a
carbon tax that will increase the cost for transportation and for
disposal by incineration. It is estimated that the additional costs will
be:

-   tipping fee for the WTE facility will increase to\$75/Mg; and
-   transportation costs will increase to \$2/Mg-km.

In this context, the cities are considering adding another landfill and
want to know if this would be cost-effective compared to using the
current facilities with the carbon tax. This landfill would have a
maximum capacity of 100 Mg/day and would be located with the following
distances from the existing sites (excluding LF1):

| **City/Facility** | **Distance to LF2 (km)** |
|:-----------------:|:------------------------:|
|         1         |            45            |
|         2         |            35            |
|         3         |            15            |
|        MRF        |            35            |
|        WTE        |            50            |

The fixed cost of operating this facility would be the same as the first
landfill, but the tipping cost would be increased to \$60/Mg-day.

#### Problem 2.1 (5 points)

What changes are needed to your optimization program from Problem 1 for
this decision problem? Formulate any different variables, objectives,
and/or constraints.

#### Problem 2.2 (3 points)

Implement the new optimization problem in `JuMP`.

#### Problem 2.3 (5 points)

Find the optimal solution and report the optimal objective value.
Provide a diagram showing the new waste flows.

#### Problem 2.4 (5 points)

Would you recommend that the cities build the new landfill? Why or why
not? Your answer should be based on your analysis but can draw on other
considerations as appropriate or desired.

## References

List any external references consulted, including classmates.